In [11]:
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import keras

# loading data
train = pd.read_csv('digit-recognizer/train.csv')
test = pd.read_csv('digit-recognizer/test.csv')

# Split features and labels in training set
X = train.drop(labels = ['label'], axis = 1)
y = train['label']
# One hot encode labels
Y = keras.utils.to_categorical(y, num_classes = 10)
# Normalize pixel data
X = X/255.0
test = test/255.0
# Reshape image in 3 dimensions (height = 28px, width = 28px , canal = 1)
X = X.values.reshape(-1, 28, 28, 1)
test = test.values.reshape(-1, 28, 28, 1)

from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from keras.optimizers.legacy import RMSprop

# Split the data in training and test set
X_train, X_test, Y_train, Y_test = train_test_split(X, Y)

model = Sequential([
         Conv2D(32, (3,3), activation='relu', input_shape=(28,28,1)),
         MaxPooling2D(2, 2),
         Conv2D(64, (3,3), activation='relu'),
         MaxPooling2D(2,2),
         Flatten(),
         Dense(512, activation='relu'),
         Dense(10, activation='softmax')
])

# Compiling model
model.compile(optimizer=RMSprop(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

# Fitting model
history = model.fit(X_train, Y_train, batch_size=100, epochs = 1)

# Predict results
results = model.predict(test)
# One hot decoding: Select the index with the maximum probability
results = np.argmax(results, axis = 1)
# Add columns ImageID and Label
results = pd.Series(results, name="Label")
submission = pd.concat([pd.Series(range(1, len(results)+1), name = "ImageId"), results], axis = 1)
# Generate CSV file
submission.to_csv('digit-recognizer/submission.csv', index=False)

875/875 [==============================] - 2s 2ms/step
